In [21]:
# rough skeleton for our main ftn

from tdmclient import ClientAsync, aw
from cv import *
from kalman import *
from motion_control import *
from utils import *
from path_planning import *
# and other imports

# repeated timer to get data from thymio(velocity, acceleration, proximity sensor)
from threading import Timer
from collections import deque
class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False


def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

def get_data():
    aw(node.wait_for_variables({"motor.left.speed","motor.right.speed", "acc"}))
    return [node["motor.left.speed"],
            node["motor.right.speed"],
            list(node["acc"])]

In [44]:
def simple_controller(x,y,theta, target_x, target_y):
    dx = target_x - x
    dy = target_y - y
    desired_theta = np.arctan2(dy, dx)
    dtheta = (desired_theta - theta + np.pi)% (2*np.pi) - np.pi
    if abs(dtheta) > 5*np.pi / 180: # abs(dtheta) larger than 3 degree
        v_ = 0
        w_ = np.sign(dtheta)*10*np.pi/180
    else:
        v_ = 1.5
        w_ = 0
    return v_, w_

In [17]:
#connect to thymio
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 310bffb3-c918-41d5-957e-fe37de1934e6

In [ ]:
dt = 0.1
epsilon = 0.7
# connect to camera, initialize map, compute global path
# initialize() function
global_path = deque()
global_path.append([0,10])
global_path.append([10,10])
global_path.append([10,0]) 
ekf = ExtendedKalmanFilter(5,5,2,dt)
camera_blocked = True
initial_state = np.array([0,0,np.pi/2,0,0])
ekf.initialize_X(initial_state)

# main loop
next_target = global_path.popleft()
print(f"next_target = {next_target[0]}, {next_target[1]}")
target_reached = False
counter = 0
while (True):
    #get data from camera(thymio_found, x, y, theta)
    thymio_found, x_camera, y_camera, theta_camera = False, 0,0,0
    camera_blocked = not thymio_found
    ekf.switch_mode(camera_blocked)

    #get data from thymio(proximity, acceleration, motor speed)
    data = get_data()
    ul = data[0]
    ur = data[1]
    acc_z = data[2][2]
    v,w = from_u_to_vw(ul, ur)
    u = np.array([v,w])
    z = np.array([x_camera, y_camera, theta_camera, v, w])
    #if kidnapping detected(from acceleration), compute global path again, initialize ekf again, continue;
    if abs(acc_z - 22) > 3: # sudden change in acc_z indicates kidnapping
        print("kidnapping detected")
        node.send_set_variables(motors(0,0))
        await client.sleep(3)
        # call initialize()
        continue

    ekf.predict_and_update(u,z)
    x,y,theta,v,w = ekf.get_X()
    if counter % 10 == 0:
        print(f"state = {x:.2f}, {y:.2f}, {theta:.2f}, {v:.2f}, {w:.2f}")
    #  check if goal is reached
    target_reached = np.linalg.norm(np.array([x - next_target[0], y - next_target[1]])) < epsilon
    if target_reached:
        if len(global_path)==0:
            print("goal reached, terminating")
            node.send_set_variables(motors(0,0))
            break
        else:
            print("heading towards next waypoint")
            next_target = global_path.popleft()
            print(f"next_target = {next_target[0]}, {next_target[1]}")

    # replace with proper motion control
    v,w = simple_controller(x,y,theta, next_target[0], next_target[1])
    ul, ur = from_vw_to_u(v,w)
    node.send_set_variables(motors(ul, ur))
    await client.sleep(dt)
    counter += 1


next_target = 0, 10
state = -0.00, -0.00, 1.57, 0.36, -0.08
state = -0.00, 0.94, 1.57, 1.53, -0.01
kidnapping detected
state = 0.02, 2.52, 1.56, 1.50, 0.01
state = 0.03, 4.04, 1.56, 1.45, -0.00
state = 0.05, 5.44, 1.56, 1.43, 0.01
state = 0.06, 6.90, 1.56, 1.37, -0.01
state = 0.07, 8.32, 1.57, 1.35, 0.01
heading towards next waypoint
next_target = 10, 10
state = 0.07, 9.66, 1.57, -0.00, -0.00
state = 0.07, 9.76, 1.44, 0.02, -0.17
state = 0.07, 9.75, 1.28, 0.02, -0.15
state = 0.08, 9.77, 1.12, -0.07, -0.18
state = 0.08, 9.77, 0.95, -0.08, -0.16
state = 0.07, 9.76, 0.79, -0.02, -0.15
state = 0.05, 9.74, 0.63, -0.02, -0.18
state = 0.02, 9.72, 0.46, -0.05, -0.17


CancelledError: 

In [43]:
 node.send_set_variables(motors(0,0))

In [15]:
await node.unlock()

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host